In [1]:
import sys
import os

from Networks.MLP import MLP
from Networks.CNN import CNN

import matplotlib.pyplot as plt
from dataset.mnist import load_mnist
from Layers.optimizer import *
from Layers.util import smooth_curve
try:
    import cupy as np
    py = 'cupy'
except ImportError:
    import numpy as np
    py = 'numpy'
import time
#


In [2]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True)
train_size = x_train.shape[0]
test_size = x_test.shape[0]
batch_size = 128
max_iterations = 200
print_iter = 10
mi = np.arange(max_iterations)

In [3]:
print(t_train)

In [4]:
def test(test_list, max_acc=None):
    networks = {}
    train_acc = {}
    train_loss = {}
    test_acc = {}
    test_loss = {}
    optimizer = {}
    total_step = {}
    time_record = {}
    print('Training with [' + py + ']\n')
    for key in test_list.keys():
        # 기록 리스트 생성
        train_acc[key] = []
        train_loss[key] = []
        test_acc[key] = []
        test_loss[key] = []
        total_step[key] = 0
        time_record[key] = 0

        # nSigmoid 경우 파라미터
        # key == 'nSigmoid_8' -> act = nSigmoid, threshold = 8
        networks[key] = test_list[key]['network']

        # optimizer 생성
        opt = test_list[key]['opt']
        lr = test_list[key]['lr']
        if opt == 'SGD':
            optimizer[key] = SGD(lr)
        elif opt == 'Momentum':
            optimizer[key] = Momentum(lr)
        elif opt == 'AdaGrad':
            optimizer[key] = AdaGrad(lr)
        elif opt == 'Adam':
            optimizer[key] = Adam(lr)
        else:
            assert 'no opt' + opt

    # 파라미터 통일 셋팅
    # networks1 = networks['1']
    # networks2 = networks['2']
    # for k in networks1.params.keys():
    #     for i in range(networks1.params[k].shape[0]):
    #         networks1.params[k][i] = networks2.params[k][i].copy()
    # print(np.max(networks['1'].params['W1'] - networks['2'].params['W1']))
    # 학습
    for i in range(max_iterations):
        if 0 not in total_step.values():
            print("All test list training Done\nStep: " + str(i))
            break

        # mini-batch train
        batch_mask = np.random.choice(train_size, batch_size)
        x_batch = x_train[batch_mask]
        t_batch = t_train[batch_mask]

        # mini-batch test
        batch_mask_t = np.random.choice(test_size, batch_size)
        x_batch_t = x_test[batch_mask_t]
        t_batch_t = t_test[batch_mask_t]

        # print_iter 회마다 경과 출력
        if i % print_iter == 0:
            print( "="*15 + "iteration:" + str(i) + "="*15)
            print("{:^9}|{:^9}|{:^9}|{:^9}".format('model','time','acc','loss'))

        # 학습 & 추론 & 기록
        for key in test_list.keys():
            if total_step[key] != 0:
                # Max acc 에 도달해 학습이 끝난 test model
                continue
            else:
                start = time.time()
                # CV 모델 데이터 처리
                if isinstance(networks[key], CNN):
                    x_batch = x_batch.reshape(-1, 1, 28, 28)
                    x_batch_t = x_batch_t.reshape(-1, 1, 28, 28)

                # 학습(역전파)
                grads = networks[key].gradient(x_batch, t_batch)
                optimizer[key].update(networks[key].params, grads)

                # 추론(순전파)
                tr_acc, tr_loss = networks[key].acc_and_loss(x_batch, t_batch)
                ts_acc, ts_loss = networks[key].acc_and_loss(x_batch_t, t_batch_t)

                # 기록
                if py == 'cupy':
                    tr_acc = tr_acc.get()
                    tr_loss = tr_loss.get()
                    ts_acc = ts_acc.get()
                    ts_loss = ts_loss.get()
                train_acc[key].append(tr_acc)
                train_loss[key].append(tr_loss)
                test_acc[key].append(ts_acc)
                test_loss[key].append(ts_loss)
                end = time.time()
                time_record[key] += (end-start)

                # max accuracy 도달 해당 모델 학습 종료
                if max_acc and max_acc <= ts_acc:
                    total_step[key] = i
                    print(key + " training end!\nacc : " + str(ts_acc) + " step: " + str(i))

                # print_iter 회마다 경과 출력
                if i % print_iter == 0:
                    print("{:^9}| {:0<7.3f} | {:0<.5f} | {:0<.5f}".format(key, time_record[key], ts_acc, ts_loss))

    return train_acc, train_loss, test_acc, test_loss, total_step


def plot(label, datas, t_list, *y_lim):
    for key in t_list:
        plt.plot(smooth_curve(datas[key]), markevery=50, label=key)
    plt.xlabel("iterations")
    plt.ylabel(label)
    if y_lim:
        plt.ylim(y_lim)
    plt.legend()
    plt.show()

In [5]:
net1 = MLP(input_size=784, hidden_size_list=[100, 100, 100, 100], output_size=10,
           use_dropout=False, dropout_ratio=0.5, use_batchnorm=True)
# net1 = MLP(input_size=784, hidden_size_list=[100, 100, 100, 100], output_size=10,
#             use_dropout=True, dropout_ratio=0.5, use_batchnorm=False)
net2 = CNN(model='CNN3', use_batchnorm=True)
net3 = CNN(model='CNN8', use_batchnorm=True)
# net3 = CNN(model='VGG11', use_batchnorm=True)
# net4 = CNN(dropout_ratio=0.5, use_batchnorm=True)

In [7]:
test_list = {
             # 'MLP':
             #     {'network':net1,'opt':'AdaGrad','lr':0.01},
             # 'SCV':
             #     {'network':net2,'opt':'AdaGrad','lr':0.01},
             'CNN':
                 {'network':net2,'opt':'AdaGrad','lr':0.01},
             # 'CNN_both':
             #     {'network':net4,'opt':'AdaGrad','lr':0.01},
             }

# train_acc, train_loss, test_acc, test_loss, total_step = test(test_list)
train_acc, train_loss, test_acc, test_loss, total_step = test(test_list, 0.98)

Training with [cupy]

===============iteration:0===============
  model  |  time   |   acc   |  loss   
   CNN   | 8.73100 | 0.12500 | 12.39902
===============iteration:10===============
  model  |  time   |   acc   |  loss   
   CNN   | 9.89700 | 0.51562 | 7.02406
===============iteration:20===============
  model  |  time   |   acc   |  loss   
   CNN   | 11.0600 | 0.60938 | 5.88260
===============iteration:30===============
  model  |  time   |   acc   |  loss   
   CNN   | 12.2150 | 0.55469 | 6.46040
===============iteration:40===============
  model  |  time   |   acc   |  loss   
   CNN   | 13.3210 | 0.53906 | 6.90243
===============iteration:50===============
  model  |  time   |   acc   |  loss   
   CNN   | 14.4440 | 0.64844 | 5.36304
===============iteration:60===============
  model  |  time   |   acc   |  loss   
   CNN   | 15.5720 | 0.57031 | 6.46090
===============iteration:70===============
  model  |  time   |   acc   |  loss   
   CNN   | 16.6770 | 0.58594 | 6.55472
==

In [ ]:
plot('test_acc', test_acc , test_list)

In [ ]:
plot('test_loss', test_loss, test_list)

In [ ]:
print(total_step)